In [10]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Function to load and preprocess images with data augmentation
def load_augmented_images(base_path):
    augmented_image_data = []
    labels = []

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for actor_id in range(1, 25):
        actor_gender = "female" if actor_id % 2 == 0 else "male"

        for emotion_id in range(1, 9):
            for intensity_id in range(1, 3):
                for statement_id in range(1, 3):
                    for repetition_id in range(1, 3):
                        filename = f"03-01-{emotion_id:02d}-{intensity_id:02d}-{statement_id:02d}-{repetition_id:02d}-{actor_id:02d}"
                        file_path = os.path.join(base_path, f"Actor_{actor_id:02d}", filename, "mono_data-channel-frequency.png")

                        # Check if the file exists before attempting to load
                        if not os.path.isfile(file_path):
                            print(f"Warning: File not found at path: {file_path}. Skipping...")
                            continue

                        # Load the image
                        img = image.load_img(file_path, target_size=(img_height, img_width), color_mode="grayscale")
                        img_array = image.img_to_array(img)
                        img_array /= 255.0  # Normalize pixel values to [0, 1]

                        # Append the original image and label
                        augmented_image_data.append(img_array)
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

                        # Augment the image and append it to the dataset
                        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape to (1, height, width, channels)
                        augmented_images = datagen.flow(img_array, batch_size=1)
                        augmented_image_data.append(augmented_images[0][0])

                        # Append the label for the augmented image
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

    return np.array(augmented_image_data), np.array(labels)

# Set the base path to the directory containing the spectrogram images
base_path = "Downloads/Saved"
img_height = 64
img_width = 64
img_channels = 1  # Assuming grayscale images

# Load and preprocess the augmented dataset
augmented_image_data, labels = load_augmented_images(base_path)

# Check if there are valid samples in the dataset
if len(augmented_image_data) == 0 or len(labels) == 0:
    print("Error: No valid samples found in the dataset.")
else:
    # Convert labels to one-hot encoding
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_onehot = to_categorical(labels_encoded, num_classes=8)  # Assuming 8 emotion classes

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(augmented_image_data, labels_onehot, test_size=0.2, random_state=42)

    if len(X_train) == 0 or len(X_test) == 0:
        print("Error: Empty training or testing sets. Adjust parameters or provide a valid dataset.")
    else:
        # Build the CNN-BiLSTM model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Reshape((-1, 64)))
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Bidirectional(LSTM(64)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(8, activation='softmax'))  # Assuming 8 emotion classes
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Define early stopping criteria
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the model with early stopping
        model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

        # Evaluate the model on the test set
        test_loss, test_accuracy = model.evaluate(X_test, y_test)
        print(f"Test Accuracy: {test_accuracy}")


Epoch 1/100
58/58 [==============================] - 17s 212ms/step - loss: 2.0704 - accuracy: 0.1270 - val_loss: 2.0661 - val_accuracy: 0.1518
Epoch 2/100
58/58 [==============================] - 11s 193ms/step - loss: 2.0635 - accuracy: 0.1221 - val_loss: 2.0692 - val_accuracy: 0.1388
Epoch 3/100
58/58 [==============================] - 11s 193ms/step - loss: 2.0609 - accuracy: 0.1335 - val_loss: 2.0597 - val_accuracy: 0.1757
Epoch 4/100
58/58 [==============================] - 11s 189ms/step - loss: 2.0595 - accuracy: 0.1487 - val_loss: 2.0605 - val_accuracy: 0.1302
Epoch 5/100
58/58 [==============================] - 11s 191ms/step - loss: 2.0517 - accuracy: 0.1438 - val_loss: 2.0497 - val_accuracy: 0.1345
Epoch 6/100
58/58 [==============================] - 11s 187ms/step - loss: 2.0246 - accuracy: 0.1910 - val_loss: 2.0489 - val_accuracy: 0.1562
Epoch 7/100
58/58 [==============================] - 11s 187ms/step - loss: 1.9915 - accuracy: 0.2143 - val_loss: 2.0007 - val_accuracy:

In [11]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Function to load and preprocess images with data augmentation
def load_augmented_images(base_path):
    augmented_image_data = []
    labels = []

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for actor_id in range(1, 25):
        actor_gender = "female" if actor_id % 2 == 0 else "male"

        for emotion_id in range(1, 9):
            for intensity_id in range(1, 3):
                for statement_id in range(1, 3):
                    for repetition_id in range(1, 3):
                        filename = f"03-01-{emotion_id:02d}-{intensity_id:02d}-{statement_id:02d}-{repetition_id:02d}-{actor_id:02d}"
                        file_path = os.path.join(base_path, f"Actor_{actor_id:02d}", filename, "mono_data-channel-frequency.png")

                        # Check if the file exists before attempting to load
                        if not os.path.isfile(file_path):
                            print(f"Warning: File not found at path: {file_path}. Skipping...")
                            continue

                        # Load the image
                        img = image.load_img(file_path, target_size=(img_height, img_width), color_mode="grayscale")
                        img_array = image.img_to_array(img)
                        img_array /= 255.0  # Normalize pixel values to [0, 1]

                        # Append the original image and label
                        augmented_image_data.append(img_array)
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

                        # Augment the image and append it to the dataset
                        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape to (1, height, width, channels)
                        augmented_images = datagen.flow(img_array, batch_size=1)
                        augmented_image_data.append(augmented_images[0][0])

                        # Append the label for the augmented image
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

    return np.array(augmented_image_data), np.array(labels)

# Set the base path to the directory containing the spectrogram images
base_path = "Downloads/Saved"
img_height = 64
img_width = 64
img_channels = 1  # Assuming grayscale images

# Load and preprocess the augmented dataset
augmented_image_data, labels = load_augmented_images(base_path)

# Check if there are valid samples in the dataset
if len(augmented_image_data) == 0 or len(labels) == 0:
    print("Error: No valid samples found in the dataset.")
else:
    # Convert labels to one-hot encoding
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_onehot = to_categorical(labels_encoded, num_classes=8)  # Assuming 8 emotion classes

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(augmented_image_data, labels_onehot, test_size=0.2, random_state=42)

    if len(X_train) == 0 or len(X_test) == 0:
        print("Error: Empty training or testing sets. Adjust parameters or provide a valid dataset.")
    else:
        # Build the CNN-BiLSTM model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Reshape((-1, 64)))
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Bidirectional(LSTM(64)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(8, activation='softmax'))  # Assuming 8 emotion classes
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Define early stopping criteria
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the model with early stopping
        model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

        # Evaluate the model on the test set
        test_loss, test_accuracy = model.evaluate(X_test, y_test)
        print(f"Test Accuracy: {test_accuracy}")


Epoch 1/100
58/58 [==============================] - 17s 209ms/step - loss: 2.0650 - accuracy: 0.1367 - val_loss: 2.0723 - val_accuracy: 0.1193
Epoch 2/100
58/58 [==============================] - 11s 188ms/step - loss: 2.0604 - accuracy: 0.1394 - val_loss: 2.0602 - val_accuracy: 0.1345
Epoch 3/100
58/58 [==============================] - 11s 187ms/step - loss: 2.0490 - accuracy: 0.1508 - val_loss: 2.0846 - val_accuracy: 0.1453
Epoch 4/100
58/58 [==============================] - 11s 187ms/step - loss: 2.0368 - accuracy: 0.1617 - val_loss: 2.0493 - val_accuracy: 0.1562
Epoch 5/100
58/58 [==============================] - 11s 190ms/step - loss: 1.9967 - accuracy: 0.2067 - val_loss: 1.9961 - val_accuracy: 0.1931
Epoch 6/100
58/58 [==============================] - 10s 180ms/step - loss: 1.9446 - accuracy: 0.2355 - val_loss: 2.0755 - val_accuracy: 0.1996
Epoch 7/100
58/58 [==============================] - 10s 180ms/step - loss: 1.9592 - accuracy: 0.2382 - val_loss: 2.0224 - val_accuracy:

In [12]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Function to load and preprocess images with data augmentation
def load_augmented_images(base_path):
    augmented_image_data = []
    labels = []

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for actor_id in range(1, 25):
        actor_gender = "female" if actor_id % 2 == 0 else "male"

        for emotion_id in range(1, 9):
            for intensity_id in range(1, 3):
                for statement_id in range(1, 3):
                    for repetition_id in range(1, 3):
                        filename = f"03-01-{emotion_id:02d}-{intensity_id:02d}-{statement_id:02d}-{repetition_id:02d}-{actor_id:02d}"
                        file_path = os.path.join(base_path, f"Actor_{actor_id:02d}", filename, "mono_data-channel-frequency.png")

                        # Check if the file exists before attempting to load
                        if not os.path.isfile(file_path):
                            print(f"Warning: File not found at path: {file_path}. Skipping...")
                            continue

                        # Load the image
                        img = image.load_img(file_path, target_size=(img_height, img_width), color_mode="grayscale")
                        img_array = image.img_to_array(img)
                        img_array /= 255.0  # Normalize pixel values to [0, 1]

                        # Append the original image and label
                        augmented_image_data.append(img_array)
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

                        # Augment the image and append it to the dataset
                        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape to (1, height, width, channels)
                        augmented_images = datagen.flow(img_array, batch_size=1)
                        augmented_image_data.append(augmented_images[0][0])

                        # Append the label for the augmented image
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

    return np.array(augmented_image_data), np.array(labels)

# Set the base path to the directory containing the spectrogram images
base_path = "Downloads/Saved"
img_height = 64
img_width = 64
img_channels = 1  # Assuming grayscale images

# Load and preprocess the augmented dataset
augmented_image_data, labels = load_augmented_images(base_path)

# Check if there are valid samples in the dataset
if len(augmented_image_data) == 0 or len(labels) == 0:
    print("Error: No valid samples found in the dataset.")
else:
    # Convert labels to one-hot encoding
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_onehot = to_categorical(labels_encoded, num_classes=8)  # Assuming 8 emotion classes

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(augmented_image_data, labels_onehot, test_size=0.2, random_state=42)

    if len(X_train) == 0 or len(X_test) == 0:
        print("Error: Empty training or testing sets. Adjust parameters or provide a valid dataset.")
    else:
        # Build the CNN-BiLSTM model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Reshape((-1, 64)))
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Bidirectional(LSTM(64)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(8, activation='softmax'))  # Assuming 8 emotion classes
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model for 100 epochs
        model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

        # Evaluate the model on the test set
        test_loss, test_accuracy = model.evaluate(X_test, y_test)
        print(f"Test Accuracy: {test_accuracy}")


Epoch 1/100
58/58 [==============================] - 17s 204ms/step - loss: 2.0660 - accuracy: 0.1416 - val_loss: 2.0773 - val_accuracy: 0.1193
Epoch 2/100
58/58 [==============================] - 11s 188ms/step - loss: 2.0583 - accuracy: 0.1378 - val_loss: 2.0724 - val_accuracy: 0.1453
Epoch 3/100
58/58 [==============================] - 11s 187ms/step - loss: 2.0574 - accuracy: 0.1492 - val_loss: 2.0592 - val_accuracy: 0.1540
Epoch 4/100
58/58 [==============================] - 11s 185ms/step - loss: 2.0261 - accuracy: 0.1796 - val_loss: 2.0110 - val_accuracy: 0.1952
Epoch 5/100
58/58 [==============================] - 11s 181ms/step - loss: 1.9990 - accuracy: 0.2013 - val_loss: 2.0148 - val_accuracy: 0.1974
Epoch 6/100
58/58 [==============================] - 10s 180ms/step - loss: 1.9696 - accuracy: 0.2132 - val_loss: 1.9747 - val_accuracy: 0.2039
Epoch 7/100
58/58 [==============================] - 10s 180ms/step - loss: 1.9568 - accuracy: 0.2241 - val_loss: 2.0160 - val_accuracy:

In [14]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Function to load and preprocess images with data augmentation
def load_augmented_images(base_path):
    augmented_image_data = []
    labels = []

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for actor_id in range(1, 25):
        actor_gender = "female" if actor_id % 2 == 0 else "male"

        for emotion_id in range(1, 9):
            for intensity_id in range(1, 3):
                for statement_id in range(1, 3):
                    for repetition_id in range(1, 3):
                        filename = f"03-01-{emotion_id:02d}-{intensity_id:02d}-{statement_id:02d}-{repetition_id:02d}-{actor_id:02d}"
                        file_path = os.path.join(base_path, f"Actor_{actor_id:02d}", filename, "mono_data-channel-frequency.png")

                        # Check if the file exists before attempting to load
                        if not os.path.isfile(file_path):
                            print(f"Warning: File not found at path: {file_path}. Skipping...")
                            continue

                        # Load the image
                        img = image.load_img(file_path, target_size=(img_height, img_width), color_mode="grayscale")
                        img_array = image.img_to_array(img)
                        img_array /= 255.0  # Normalize pixel values to [0, 1]

                        # Append the original image and label
                        augmented_image_data.append(img_array)
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

                        # Augment the image and append it to the dataset
                        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape to (1, height, width, channels)
                        augmented_images = datagen.flow(img_array, batch_size=1)
                        augmented_image_data.append(augmented_images[0][0])

                        # Append the label for the augmented image
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

    return np.array(augmented_image_data), np.array(labels)

# Set the base path to the directory containing the spectrogram images
base_path = "Downloads/Saved"
img_height = 64
img_width = 64
img_channels = 1  # Assuming grayscale images

# Load and preprocess the augmented dataset
augmented_image_data, labels = load_augmented_images(base_path)

# Check if there are valid samples in the dataset
if len(augmented_image_data) == 0 or len(labels) == 0:
    print("Error: No valid samples found in the dataset.")
else:
    # Convert labels to one-hot encoding
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_onehot = to_categorical(labels_encoded, num_classes=8)  # Assuming 8 emotion classes

    # Split the dataset into training and testing sets with a 20% test size
    X_train, X_test, y_train, y_test = train_test_split(augmented_image_data, labels_onehot, test_size=0.2, random_state=42)

    if len(X_train) == 0 or len(X_test) == 0:
        print("Error: Empty training or testing sets. Adjust parameters or provide a valid dataset.")
    else:
        # Build the CNN-BiLSTM model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Reshape((-1, 64)))
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Bidirectional(LSTM(64)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(8, activation='softmax'))  # Assuming 8 emotion classes
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Define early stopping criteria
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the model with early stopping
        model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

        # Evaluate the model on the test set
        test_loss, test_accuracy = model.evaluate(X_test, y_test)
        print(f"Test Accuracy: {test_accuracy}")


Epoch 1/100
58/58 [==============================] - 19s 230ms/step - loss: 2.0681 - accuracy: 0.1356 - val_loss: 2.0727 - val_accuracy: 0.1302
Epoch 2/100
58/58 [==============================] - 11s 190ms/step - loss: 2.0596 - accuracy: 0.1373 - val_loss: 2.0674 - val_accuracy: 0.1432
Epoch 3/100
58/58 [==============================] - 11s 186ms/step - loss: 2.0569 - accuracy: 0.1584 - val_loss: 2.0715 - val_accuracy: 0.1323
Epoch 4/100
58/58 [==============================] - 11s 187ms/step - loss: 2.0646 - accuracy: 0.1243 - val_loss: 2.0672 - val_accuracy: 0.1497
Epoch 5/100
58/58 [==============================] - 11s 186ms/step - loss: 2.0584 - accuracy: 0.1422 - val_loss: 2.0544 - val_accuracy: 0.1605
Epoch 6/100
58/58 [==============================] - 11s 186ms/step - loss: 2.0605 - accuracy: 0.1422 - val_loss: 2.0648 - val_accuracy: 0.1670
Epoch 7/100
58/58 [==============================] - 11s 189ms/step - loss: 2.0493 - accuracy: 0.1677 - val_loss: 2.0694 - val_accuracy:

In [15]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Function to load and preprocess images with data augmentation
def load_augmented_images(base_path):
    augmented_image_data = []
    labels = []

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for actor_id in range(1, 25):
        actor_gender = "female" if actor_id % 2 == 0 else "male"

        for emotion_id in range(1, 9):
            for intensity_id in range(1, 3):
                for statement_id in range(1, 3):
                    for repetition_id in range(1, 3):
                        filename = f"03-01-{emotion_id:02d}-{intensity_id:02d}-{statement_id:02d}-{repetition_id:02d}-{actor_id:02d}"
                        file_path = os.path.join(base_path, f"Actor_{actor_id:02d}", filename, "mono_data-channel-frequency.png")

                        # Check if the file exists before attempting to load
                        if not os.path.isfile(file_path):
                            print(f"Warning: File not found at path: {file_path}. Skipping...")
                            continue

                        # Load the image
                        img = image.load_img(file_path, target_size=(img_height, img_width), color_mode="grayscale")
                        img_array = image.img_to_array(img)
                        img_array /= 255.0  # Normalize pixel values to [0, 1]

                        # Append the original image and label
                        augmented_image_data.append(img_array)
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

                        # Augment the image and append it to the dataset
                        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape to (1, height, width, channels)
                        augmented_images = datagen.flow(img_array, batch_size=1)
                        augmented_image_data.append(augmented_images[0][0])

                        # Append the label for the augmented image
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

    return np.array(augmented_image_data), np.array(labels)

# Set the base path to the directory containing the spectrogram images
base_path = "Downloads/Saved"
img_height = 64
img_width = 64
img_channels = 1  # Assuming grayscale images

# Load and preprocess the augmented dataset
augmented_image_data, labels = load_augmented_images(base_path)

# Check if there are valid samples in the dataset
if len(augmented_image_data) == 0 or len(labels) == 0:
    print("Error: No valid samples found in the dataset.")
else:
    # Convert labels to one-hot encoding
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_onehot = to_categorical(labels_encoded, num_classes=8)  # Assuming 8 emotion classes

    # Split the dataset into training and testing sets with a 20% test size
    X_train, X_test, y_train, y_test = train_test_split(augmented_image_data, labels_onehot, test_size=0.2, random_state=42)

    if len(X_train) == 0 or len(X_test) == 0:
        print("Error: Empty training or testing sets. Adjust parameters or provide a valid dataset.")
    else:
        # Build the CNN-BiLSTM model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Reshape((-1, 64)))
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Bidirectional(LSTM(64)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(8, activation='softmax'))  # Assuming 8 emotion classes
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Define early stopping criteria
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the model with early stopping
        model.fit(X_train, y_train, epochs=50, batch_size=1, validation_split=0.2, callbacks=[early_stopping])

        # Evaluate the model on the test set
        test_loss, test_accuracy = model.evaluate(X_test, y_test)
        print(f"Test Accuracy: {test_accuracy}")


Epoch 1/50
1843/1843 [==============================] - 121s 63ms/step - loss: 2.0744 - accuracy: 0.1362 - val_loss: 2.0716 - val_accuracy: 0.1388
Epoch 2/50
1843/1843 [==============================] - 114s 62ms/step - loss: 2.0649 - accuracy: 0.1362 - val_loss: 2.0670 - val_accuracy: 0.1302
Epoch 3/50
1843/1843 [==============================] - 117s 63ms/step - loss: 2.0643 - accuracy: 0.1389 - val_loss: 2.0963 - val_accuracy: 0.1106
Epoch 4/50
1843/1843 [==============================] - 116s 63ms/step - loss: 2.0659 - accuracy: 0.1329 - val_loss: 2.0678 - val_accuracy: 0.1193
Epoch 5/50
1843/1843 [==============================] - 115s 63ms/step - loss: 2.0633 - accuracy: 0.1319 - val_loss: 2.0709 - val_accuracy: 0.1106
Epoch 6/50
1843/1843 [==============================] - 116s 63ms/step - loss: 2.0625 - accuracy: 0.1259 - val_loss: 2.0676 - val_accuracy: 0.1518
Epoch 7/50
1843/1843 [==============================] - 115s 63ms/step - loss: 2.0631 - accuracy: 0.1346 - val_loss: 2

In [16]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Function to load and preprocess images with data augmentation
def load_augmented_images(base_path):
    augmented_image_data = []
    labels = []

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for actor_id in range(1, 25):
        actor_gender = "female" if actor_id % 2 == 0 else "male"

        for emotion_id in range(1, 9):
            for intensity_id in range(1, 3):
                for statement_id in range(1, 3):
                    for repetition_id in range(1, 3):
                        filename = f"03-01-{emotion_id:02d}-{intensity_id:02d}-{statement_id:02d}-{repetition_id:02d}-{actor_id:02d}"
                        file_path = os.path.join(base_path, f"Actor_{actor_id:02d}", filename, "mono_data-channel-frequency.png")

                        # Check if the file exists before attempting to load
                        if not os.path.isfile(file_path):
                            print(f"Warning: File not found at path: {file_path}. Skipping...")
                            continue

                        # Load the image
                        img = image.load_img(file_path, target_size=(img_height, img_width), color_mode="grayscale")
                        img_array = image.img_to_array(img)
                        img_array /= 255.0  # Normalize pixel values to [0, 1]

                        # Append the original image and label
                        augmented_image_data.append(img_array)
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

                        # Augment the image and append it to the dataset
                        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape to (1, height, width, channels)
                        augmented_images = datagen.flow(img_array, batch_size=1)
                        augmented_image_data.append(augmented_images[0][0])

                        # Append the label for the augmented image
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

    return np.array(augmented_image_data), np.array(labels)

# Set the base path to the directory containing the spectrogram images
base_path = "Downloads/Saved"
img_height = 64
img_width = 64
img_channels = 1  # Assuming grayscale images

# Load and preprocess the augmented dataset
augmented_image_data, labels = load_augmented_images(base_path)

# Check if there are valid samples in the dataset
if len(augmented_image_data) == 0 or len(labels) == 0:
    print("Error: No valid samples found in the dataset.")
else:
    # Convert labels to one-hot encoding
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_onehot = to_categorical(labels_encoded, num_classes=8)  # Assuming 8 emotion classes

    # Split the dataset into training and testing sets with a 20% test size
    X_train, X_test, y_train, y_test = train_test_split(augmented_image_data, labels_onehot, test_size=0.2, random_state=42)

    if len(X_train) == 0 or len(X_test) == 0:
        print("Error: Empty training or testing sets. Adjust parameters or provide a valid dataset.")
    else:
        # Build the CNN-BiLSTM model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Reshape((-1, 64)))
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Bidirectional(LSTM(64)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(8, activation='softmax'))  # Assuming 8 emotion classes
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Define early stopping criteria
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the model with early stopping
        model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

        # Evaluate the model on the test set
        test_loss, test_accuracy = model.evaluate(X_test, y_test)
        print(f"Test Accuracy: {test_accuracy}")


Epoch 1/50
29/29 [==============================] - 16s 380ms/step - loss: 2.0689 - accuracy: 0.1226 - val_loss: 2.0650 - val_accuracy: 0.1302
Epoch 2/50
29/29 [==============================] - 10s 335ms/step - loss: 2.0596 - accuracy: 0.1432 - val_loss: 2.0726 - val_accuracy: 0.1106
Epoch 3/50
29/29 [==============================] - 11s 383ms/step - loss: 2.0567 - accuracy: 0.1367 - val_loss: 2.0627 - val_accuracy: 0.1388
Epoch 4/50
29/29 [==============================] - 11s 378ms/step - loss: 2.0481 - accuracy: 0.1503 - val_loss: 2.0613 - val_accuracy: 0.1388
Epoch 5/50
29/29 [==============================] - 11s 378ms/step - loss: 2.0422 - accuracy: 0.1525 - val_loss: 2.0472 - val_accuracy: 0.1649
Epoch 6/50
29/29 [==============================] - 11s 379ms/step - loss: 2.0106 - accuracy: 0.1899 - val_loss: 2.0476 - val_accuracy: 0.1605
Epoch 7/50
29/29 [==============================] - 11s 376ms/step - loss: 2.0494 - accuracy: 0.1584 - val_loss: 2.0318 - val_accuracy: 0.1605

In [17]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Function to load and preprocess images with data augmentation
def load_augmented_images(base_path):
    augmented_image_data = []
    labels = []

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for actor_id in range(1, 25):
        actor_gender = "female" if actor_id % 2 == 0 else "male"

        for emotion_id in range(1, 9):
            for intensity_id in range(1, 3):
                for statement_id in range(1, 3):
                    for repetition_id in range(1, 3):
                        filename = f"03-01-{emotion_id:02d}-{intensity_id:02d}-{statement_id:02d}-{repetition_id:02d}-{actor_id:02d}"
                        file_path = os.path.join(base_path, f"Actor_{actor_id:02d}", filename, "mono_data-channel-frequency.png")

                        # Check if the file exists before attempting to load
                        if not os.path.isfile(file_path):
                            print(f"Warning: File not found at path: {file_path}. Skipping...")
                            continue

                        # Load the image
                        img = image.load_img(file_path, target_size=(img_height, img_width), color_mode="grayscale")
                        img_array = image.img_to_array(img)
                        img_array /= 255.0  # Normalize pixel values to [0, 1]

                        # Append the original image and label
                        augmented_image_data.append(img_array)
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

                        # Augment the image and append it to the dataset
                        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape to (1, height, width, channels)
                        augmented_images = datagen.flow(img_array, batch_size=1)
                        augmented_image_data.append(augmented_images[0][0])

                        # Append the label for the augmented image
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

    return np.array(augmented_image_data), np.array(labels)

# Set the base path to the directory containing the spectrogram images
base_path = "Downloads/Saved"
img_height = 64
img_width = 64
img_channels = 1  # Assuming grayscale images

# Load and preprocess the augmented dataset
augmented_image_data, labels = load_augmented_images(base_path)

# Check if there are valid samples in the dataset
if len(augmented_image_data) == 0 or len(labels) == 0:
    print("Error: No valid samples found in the dataset.")
else:
    # Convert labels to one-hot encoding
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_onehot = to_categorical(labels_encoded, num_classes=8)  # Assuming 8 emotion classes

    # Split the dataset into training and testing sets with a 20% test size
    X_train, X_test, y_train, y_test = train_test_split(augmented_image_data, labels_onehot, test_size=0.2, random_state=42)

    if len(X_train) == 0 or len(X_test) == 0:
        print("Error: Empty training or testing sets. Adjust parameters or provide a valid dataset.")
    else:
        # Build the CNN-BiLSTM model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Reshape((-1, 64)))
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Bidirectional(LSTM(64)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(8, activation='softmax'))  # Assuming 8 emotion classes
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Define early stopping criteria
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the model with early stopping
        model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2, callbacks=[early_stopping])

        # Evaluate the model on the test set
        test_loss, test_accuracy = model.evaluate(X_test, y_test)
        print(f"Test Accuracy: {test_accuracy}")


Epoch 1/50
185/185 [==============================] - 25s 107ms/step - loss: 2.0696 - accuracy: 0.1253 - val_loss: 2.0639 - val_accuracy: 0.1453
Epoch 2/50
185/185 [==============================] - 19s 101ms/step - loss: 2.0626 - accuracy: 0.1291 - val_loss: 2.0623 - val_accuracy: 0.1367
Epoch 3/50
185/185 [==============================] - 19s 101ms/step - loss: 2.0511 - accuracy: 0.1606 - val_loss: 2.0626 - val_accuracy: 0.1280
Epoch 4/50
185/185 [==============================] - 19s 101ms/step - loss: 2.0191 - accuracy: 0.1948 - val_loss: 2.0163 - val_accuracy: 0.1562
Epoch 5/50
185/185 [==============================] - 19s 100ms/step - loss: 2.0107 - accuracy: 0.1926 - val_loss: 2.0375 - val_accuracy: 0.1757
Epoch 6/50
185/185 [==============================] - 19s 100ms/step - loss: 1.9732 - accuracy: 0.2116 - val_loss: 2.0025 - val_accuracy: 0.1822
Epoch 7/50
185/185 [==============================] - 19s 101ms/step - loss: 1.9535 - accuracy: 0.2306 - val_loss: 1.9952 - val_ac

In [18]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Function to load and preprocess images with data augmentation
def load_augmented_images(base_path):
    augmented_image_data = []
    labels = []

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for actor_id in range(1, 25):
        actor_gender = "female" if actor_id % 2 == 0 else "male"

        for emotion_id in range(1, 9):
            for intensity_id in range(1, 3):
                for statement_id in range(1, 3):
                    for repetition_id in range(1, 3):
                        filename = f"03-01-{emotion_id:02d}-{intensity_id:02d}-{statement_id:02d}-{repetition_id:02d}-{actor_id:02d}"
                        file_path = os.path.join(base_path, f"Actor_{actor_id:02d}", filename, "mono_data-channel-frequency.png")

                        # Check if the file exists before attempting to load
                        if not os.path.isfile(file_path):
                            print(f"Warning: File not found at path: {file_path}. Skipping...")
                            continue

                        # Load the image
                        img = image.load_img(file_path, target_size=(img_height, img_width), color_mode="grayscale")
                        img_array = image.img_to_array(img)
                        img_array /= 255.0  # Normalize pixel values to [0, 1]

                        # Append the original image and label
                        augmented_image_data.append(img_array)
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

                        # Augment the image and append it to the dataset
                        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape to (1, height, width, channels)
                        augmented_images = datagen.flow(img_array, batch_size=1)
                        augmented_image_data.append(augmented_images[0][0])

                        # Append the label for the augmented image
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

    return np.array(augmented_image_data), np.array(labels)

# Set the base path to the directory containing the spectrogram images
base_path = "Downloads/Saved"
img_height = 64
img_width = 64
img_channels = 1  # Assuming grayscale images

# Load and preprocess the augmented dataset
augmented_image_data, labels = load_augmented_images(base_path)

# Check if there are valid samples in the dataset
if len(augmented_image_data) == 0 or len(labels) == 0:
    print("Error: No valid samples found in the dataset.")
else:
    # Convert labels to one-hot encoding
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_onehot = to_categorical(labels_encoded, num_classes=8)  # Assuming 8 emotion classes

    # Split the dataset into training and testing sets with a 20% test size
    X_train, X_test, y_train, y_test = train_test_split(augmented_image_data, labels_onehot, test_size=0.2, random_state=42)

    if len(X_train) == 0 or len(X_test) == 0:
        print("Error: Empty training or testing sets. Adjust parameters or provide a valid dataset.")
    else:
        # Build the CNN-BiLSTM model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Reshape((-1, 64)))
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Bidirectional(LSTM(64)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(8, activation='softmax'))  # Assuming 8 emotion classes
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Define early stopping criteria
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the model with early stopping
        model.fit(X_train, y_train, epochs=100, batch_size=5, validation_split=0.2, callbacks=[early_stopping])

        # Evaluate the model on the test set
        test_loss, test_accuracy = model.evaluate(X_test, y_test)
        print(f"Test Accuracy: {test_accuracy}")


Epoch 1/100
369/369 [==============================] - 40s 93ms/step - loss: 2.0713 - accuracy: 0.1226 - val_loss: 2.0667 - val_accuracy: 0.1193
Epoch 2/100
369/369 [==============================] - 33s 88ms/step - loss: 2.0642 - accuracy: 0.1281 - val_loss: 2.0651 - val_accuracy: 0.1323
Epoch 3/100
369/369 [==============================] - 33s 88ms/step - loss: 2.0534 - accuracy: 0.1568 - val_loss: 2.0299 - val_accuracy: 0.1974
Epoch 4/100
369/369 [==============================] - 33s 88ms/step - loss: 2.0194 - accuracy: 0.1834 - val_loss: 2.0188 - val_accuracy: 0.1909
Epoch 5/100
369/369 [==============================] - 33s 89ms/step - loss: 1.9991 - accuracy: 0.2067 - val_loss: 2.0279 - val_accuracy: 0.1757
Epoch 6/100
369/369 [==============================] - 33s 89ms/step - loss: 1.9803 - accuracy: 0.2214 - val_loss: 1.9789 - val_accuracy: 0.2343
Epoch 7/100
369/369 [==============================] - 33s 89ms/step - loss: 1.9582 - accuracy: 0.2192 - val_loss: 1.9779 - val_ac

In [19]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Function to load and preprocess images with data augmentation
def load_augmented_images(base_path):
    augmented_image_data = []
    labels = []

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for actor_id in range(1, 25):
        actor_gender = "female" if actor_id % 2 == 0 else "male"

        for emotion_id in range(1, 9):
            for intensity_id in range(1, 3):
                for statement_id in range(1, 3):
                    for repetition_id in range(1, 3):
                        filename = f"03-01-{emotion_id:02d}-{intensity_id:02d}-{statement_id:02d}-{repetition_id:02d}-{actor_id:02d}"
                        file_path = os.path.join(base_path, f"Actor_{actor_id:02d}", filename, "mono_data-channel-frequency.png")

                        # Check if the file exists before attempting to load
                        if not os.path.isfile(file_path):
                            print(f"Warning: File not found at path: {file_path}. Skipping...")
                            continue

                        # Load the image
                        img = image.load_img(file_path, target_size=(img_height, img_width), color_mode="grayscale")
                        img_array = image.img_to_array(img)
                        img_array /= 255.0  # Normalize pixel values to [0, 1]

                        # Append the original image and label
                        augmented_image_data.append(img_array)
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

                        # Augment the image and append it to the dataset
                        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape to (1, height, width, channels)
                        augmented_images = datagen.flow(img_array, batch_size=1)
                        augmented_image_data.append(augmented_images[0][0])

                        # Append the label for the augmented image
                        labels.append(emotion_id - 1)  # Subtract 1 to make emotion_id start from 0

    return np.array(augmented_image_data), np.array(labels)

# Set the base path to the directory containing the spectrogram images
base_path = "Downloads/Saved"
img_height = 64
img_width = 64
img_channels = 1  # Assuming grayscale images

# Load and preprocess the augmented dataset
augmented_image_data, labels = load_augmented_images(base_path)

# Check if there are valid samples in the dataset
if len(augmented_image_data) == 0 or len(labels) == 0:
    print("Error: No valid samples found in the dataset.")
else:
    # Convert labels to one-hot encoding
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_onehot = to_categorical(labels_encoded, num_classes=8)  # Assuming 8 emotion classes

    # Split the dataset into training and testing sets with a 20% test size
    X_train, X_test, y_train, y_test = train_test_split(augmented_image_data, labels_onehot, test_size=0.2, random_state=42)

    if len(X_train) == 0 or len(X_test) == 0:
        print("Error: Empty training or testing sets. Adjust parameters or provide a valid dataset.")
    else:
        # Build the CNN-BiLSTM model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Reshape((-1, 64)))
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Bidirectional(LSTM(64)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(8, activation='softmax'))  # Assuming 8 emotion classes
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Define early stopping criteria
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the model with early stopping
        model.fit(X_train, y_train, epochs=100, batch_size=100, validation_split=0.2, callbacks=[early_stopping])

        # Evaluate the model on the test set
        test_loss, test_accuracy = model.evaluate(X_test, y_test)
        print(f"Test Accuracy: {test_accuracy}")


Epoch 1/100
19/19 [==============================] - 16s 591ms/step - loss: 2.0712 - accuracy: 0.1432 - val_loss: 2.0669 - val_accuracy: 0.1453
Epoch 2/100
19/19 [==============================] - 10s 527ms/step - loss: 2.0612 - accuracy: 0.1367 - val_loss: 2.0663 - val_accuracy: 0.1150
Epoch 3/100
19/19 [==============================] - 10s 524ms/step - loss: 2.0579 - accuracy: 0.1329 - val_loss: 2.0629 - val_accuracy: 0.1605
Epoch 4/100
19/19 [==============================] - 10s 534ms/step - loss: 2.0565 - accuracy: 0.1568 - val_loss: 2.0596 - val_accuracy: 0.1540
Epoch 5/100
19/19 [==============================] - 10s 525ms/step - loss: 2.0484 - accuracy: 0.1639 - val_loss: 2.0498 - val_accuracy: 0.1584
Epoch 6/100
19/19 [==============================] - 10s 534ms/step - loss: 2.0327 - accuracy: 0.1693 - val_loss: 2.0364 - val_accuracy: 0.1714
Epoch 7/100
19/19 [==============================] - 10s 528ms/step - loss: 2.0123 - accuracy: 0.1791 - val_loss: 2.0129 - val_accuracy: